In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import os
from pyspark.sql import SparkSession
from google.cloud import storage
from dotenv import load_dotenv
import json

In [2]:
spark = SparkSession.builder \
    .appName('ParquetRepartitioning') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/07/02 16:39:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
url = 'https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page'
load_dotenv()
bucket_name = os.getenv('GCS_BUCKET_NAME')

In [3]:
response = requests.get(url)
html_content = response.text
soup = BeautifulSoup(html_content, 'html.parser')
links = soup.find_all('a', title='Yellow Taxi Trip Records', href=True)
print(links[0]['href'])

https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-01.parquet 


In [4]:
years = range(2012, 2008, -1)
months = range(1, 13)
for year in years:
    for month in months:
        download_url = f'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{year}-{month:02d}.parquet' 
        filename = f'yellow_tripdata_{year}-{month:02d}.parquet'
        with requests.get(download_url, stream = True) as response:
            response.raise_for_status()
            with open(filename, 'wb') as file:
                for chunk in response.iter_content(chunk_size = 8192):
                    file.write(chunk)
        print(f'File {filename} downloaded')
        client = storage.Client()
        bucket = client.bucket(bucket_name)
        blob = bucket.blob(f'raw_pq/{filename}')
        blob.upload_from_filename(filename)
        print(f"Uploaded {filename} to GCS bucket {bucket_name}")

        os.remove(filename)
        print(f'File {filename} removed')

File yellow_tripdata_2012-01.parquet downloaded
Uploaded yellow_tripdata_2012-01.parquet to GCS bucket nyc-project-data-lake
File yellow_tripdata_2012-01.parquet removed
File yellow_tripdata_2012-02.parquet downloaded
Uploaded yellow_tripdata_2012-02.parquet to GCS bucket nyc-project-data-lake
File yellow_tripdata_2012-02.parquet removed
File yellow_tripdata_2012-03.parquet downloaded
Uploaded yellow_tripdata_2012-03.parquet to GCS bucket nyc-project-data-lake
File yellow_tripdata_2012-03.parquet removed
File yellow_tripdata_2012-04.parquet downloaded
Uploaded yellow_tripdata_2012-04.parquet to GCS bucket nyc-project-data-lake
File yellow_tripdata_2012-04.parquet removed
File yellow_tripdata_2012-05.parquet downloaded
Uploaded yellow_tripdata_2012-05.parquet to GCS bucket nyc-project-data-lake
File yellow_tripdata_2012-05.parquet removed
File yellow_tripdata_2012-06.parquet downloaded
Uploaded yellow_tripdata_2012-06.parquet to GCS bucket nyc-project-data-lake
File yellow_tripdata_2012